# 02 - Neural Network Classification with TensorFlow

**Content of this notebook :**
- Architecture of a classification model
- Input shapes and output shapes
    - `X` : features/data (inputs)
    - `y` : labels (outputs)
        - "What class do the inputs belong to?"
- Creating custom data to view and fit
- Steps in modelling for binary and multiclass classification
    - Creating a model
    - Compiling a model
        - Defining a loss function
        - Setting up an optimizer
            - Finding the best learning rate
        - Creating evaluation metrics
    - Fitting a model (getting it to find patterns in our data)
    - Improving a model
- The power of non-linearity
- Evaluating classification models
    - Visualizing the model
    - Looking at training curves
    - Compare predictions to ground truth

## What is a Classification problem?

A `classification problem` involves predicting whether something is one thing or another.

For example we might want to:
- Predict whether or not someone has heart disease based on their health parameters. This is called **binary classification** since there are only two options.
- Decide whether a photo is of food, a person or a dog. This is called **multi-class classification** sine there are more than two options.
- Predict what categories should be assigned to Wikipedia article. This is called **multi-label classification** since a single article could have more than one category assigned.

## Typical Architecture of neural network classification models with TensorFlow